In [1]:
import pandas as pd
from ch2.uranus.route.cda import *
from ch2.data import *
from ch2.squeal.database import connect

_, log, db = connect(['-v 0'])
with db.session_context() as s:
    df = accumulate_cda_crr(64 + 12, 
                            expand_bookmarks(log, s,
                                             filter_bookmarks(s, 0,
                                                              read_bookmarks(s, '60s/0ms'))))
df.describe()

,CdA,Crr
count,9976,9976
unique,9976,9976
top,"(0, 2.300933072789198)","(7.256874928941692, 0)"
freq,1,1


In [2]:
from bokeh.plotting import figure, output_notebook, show

output_notebook()
f = figure(plot_width=500, plot_height=400)
f.multi_line(xs='CdA', ys='Crr', source=df, line_alpha=0.1, line_color='black')
f.xaxis.axis_label = 'CdA'
f.yaxis.axis_label = 'Crr'
show(f)

Loading BokehJS ...

In [3]:
import numpy as np

r = np.linspace(0, 1.5, 100)
w = r[1] - r[0]
cda_0 = pd.DataFrame(columns=['CdA'], data=[cda[1] for cda in df['CdA']])
bins = cda_0.groupby(pd.cut(cda_0['CdA'], r)).size()
print(bins.describe())

f = figure(plot_width=900, plot_height=300)
f.quad(top=bins, left=r[:-1]+0.1*w, right=r[1:]-0.1*w, bottom=0)
show(f)

count     99.000000
mean      47.565657
std       25.321670
min        7.000000
25%       29.500000
50%       41.000000
75%       64.500000
max      108.000000
dtype: float64


In [4]:
import scipy as sp
from scipy.optimize import fmin

cda = pd.DataFrame({'CdA': 0.5 * (r[:-1] + r[1:]), 'n': bins.values})
f = figure(plot_width=900, plot_height=300)
# f.circle(x='CdA', y='n', source=cda)
f.quad(top=bins, left=r[:-1]+0.1*w, right=r[1:]-0.1*w, bottom=0)
for order in range(2, 20, 2):
    coeff = sp.polyfit(cda['CdA'], cda['n'], order)
    p = sp.poly1d(coeff)
    print(order, fmin(lambda x: -p(x), 0.6, disp=0)[0])
    f.line(x=cda['CdA'], y=p(cda['CdA']), line_color='orange')
show(f)         


2 0.7279101562500003
4 0.6036328125
6 0.5425781249999999
8 0.5296874999999999
10 0.5254687499999997
12 0.5084179687499997
14 0.5026171874999997
16 0.4990429687499997
18 0.4982226562499997


In [5]:
with db.session_context() as s:
    df = accumulate_crr(67 + 12, 
                        expand_bookmarks(log, s,
                                         filter_bookmarks(s, 0,
                                                          read_bookmarks(s, '60s'))),
                       cda=0.55)
df.describe()

,Crr
count,0
unique,0


In [6]:
r = np.linspace(0, 100, 100)
w = r[1] - r[0]
bins = df.groupby(pd.cut(df['Crr'], r)).size()
print(bins.describe())

f = figure(plot_width=900, plot_height=300)
f.quad(top=bins, left=r[:-1]+0.1*w, right=r[1:]-0.1*w, bottom=0)
show(f)



count    99.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
